In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from datetime import datetime
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing
# from tensorflow.python.ops.gen_math_ops import mod
# from PIL import Image, ImageFilter
# import mediapipe
from matplotlib import pyplot as plt
# from plot_keras_history import plot_history

In [2]:
EPOCHS = 10
IMG_WIDTH = 256
IMG_HEIGHT = 150
NUM_CATEGORIES = 5
# TEST_SIZE = 0.4
GESTURE = {0:"ok", 1:"up", 2:"down", 3:"palm", 4:"l"}

In [3]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.
    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.
    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels = []
    
    for dir in range(0, NUM_CATEGORIES):
        # get path for each gesture like "/home/arpine/Desktop/data/0":  
        d = os.path.join(data_dir, f"{str(dir)}")
        # os.listdir(d) return the list of all names of images in that folder
        for image_path in os.listdir(d):
            # get the full path of specific image 
            full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
            # Returns an image that is loaded from the specified file
            image = cv2.imread(full_path, )
        #     print(image.shape)
        #     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #     print(image.shape)
            # cv2.imshow("im", image)
            # get dimension for each image
            dim = (IMG_WIDTH, IMG_HEIGHT)
            # resized the image
            image_resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        #     print(image_resized.shape)
            
        #     add image and their directory name to images and labels list
            images.append(image_resized)
            labels.append(dir)
    # images = np.load(images)
    # labels = np.load(labels)

    
    return images, labels

In [4]:
start_time = datetime.now()  
print("Loading ===========") 
images_test, labels_test = load_data("/home/arpine/Desktop/Gesture/test") 
images_train, labels_train = load_data("/home/arpine/Desktop/Gesture/train") 
valid_image, valid_label  = load_data("/home/arpine/Desktop/Gesture/valid")

# images_test, labels_test = load_data("images_test.npy", "labels_test.npy") 
# images_train, labels_train = load_data("images_train.npy", "labels_train.npy" ) 
# valid_image, valid_label  = load_data("images_valid.npy", "labels_valid.npy" )
finish_loading_time = datetime.now()
print("Images load time: ", finish_loading_time - start_time)

Loading ===========
Images load time:  0:03:55.789620


In [5]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # Create a convolutional neural network
    model = tf.keras.models.Sequential(
        [
        # Convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            32, (5, 5), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),
        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            64, (3, 3), activation='relu' #, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),
        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        # tf.keras.layers.Conv2D(
        #     64, (3, 3), activation='relu' #, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        # ),
        # # Max-pooling layer, using 2x2 pool size
        # tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            128, (3, 3), activation='relu' #, input_shape=((IMG_WIDTH), (IMG_HEIGHT), 3)
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        # tf.keras.layers.Conv2D(
        #     128, (3, 3), activation='relu' #, input_shape=((IMG_WIDTH), (IMG_HEIGHT), 3)
        # ),
        # tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            256, (3, 3), activation='relu'#, input_shape=((IMG_WIDTH), (IMG_HEIGHT), 3)
        ),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        
        tf.keras.layers.Flatten(),
        # Add a hidden layer with dropout
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        # tf.keras.layers.Dense(128, activation='relu'),
        # tf.keras.layers.Dropout(0.3),
        # Add an output layer with output units for all 6 gestures
        # tf.keras.layers.Dense(NUM_CATEGORIES, activation='sigmoid')
        tf.keras.layers.Dense(NUM_CATEGORIES, activation='softmax')
    ])

  
    # optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    # model.compile(
    #     optimizer=optimizer ,
    #     loss="categorical_crossentropy",
    #     metrics=["accuracy"]
    # )
    # return model

      # Train neural network
    model.compile(
        optimizer='adam',
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [6]:
print("====================")
y_test = tf.keras.utils.to_categorical(labels_test)
y_train = tf.keras.utils.to_categorical(labels_train)
y_valid = tf.keras.utils.to_categorical(valid_label)

x_train = np.array(images_train)/255

x_test = np.array(images_test)/255
# print(x_test)
x_valid = np.array(valid_image)/255

# x_test_min = np.array(images_test).min(axis=(1, 2), keepdims=True)
# x_test_max = np.array(images_test).max(axis=(1, 2), keepdims=True)

# x_test = (np.array(images_test) - x_test_min) / (x_test_max - x_test_min)
# print(x_test)

# x_train_min = np.array(images_train).min(axis=(1, 2), keepdims=True)
# x_train_max = np.array(images_train).max(axis=(1, 2), keepdims=True)

# x_train = (np.array(images_train) - x_train_min) / (x_train_max - x_train_min)
# print(x_train.shape)


# x_valid_min = np.array(valid_image).min(axis=(1, 2), keepdims=True)
# x_valid_max = np.array(valid_image).max(axis=(1, 2), keepdims=True)

# x_valid = (np.array(valid_image) - x_valid_min) / (x_valid_max - x_valid_min)
# print(x_valid.shape)


print("====================")
# Get a compiled neural network
fitting_time = datetime.now()
model = get_model()

# Fit model on training data
history = model.fit(x_train, y_train, epochs=EPOCHS, validation_data = (x_valid, y_valid))
# model.fit(train_data, epochs= EPOCHS)
# Evaluate neural network performance
model.evaluate(x_test, y_test, verbose=2)
finish_loading_time = datetime.now()

print("NN fit time: ", datetime.now() - fitting_time)
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['y_train', 'y_test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['y_train', 'y_test'], loc='upper left')
plt.show()

# x_test = preprocessing.normalize(np.array(images_test))
# x_train = preprocessing.normalize(np.array(images_train))
# x_valid = preprocessing.normalize(np.array(valid_image))



In [ ]:
# # Get a compiled neural network
# fitting_time = datetime.now()
# model = get_model()

# # Fit model on training data
# history = model.fit(x_train, y_train, epochs=EPOCHS, validation_data = (x_valid, y_valid))
# # model.fit(train_data, epochs= EPOCHS)
# # Evaluate neural network performance
# model.evaluate(x_test, y_test, verbose=2)
# finish_loading_time = datetime.now()

# print("NN fit time: ", datetime.now() - fitting_time)
# print(history.history.keys())
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['y_train', 'y_test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['y_train', 'y_test'], loc='upper left')
# plt.show()

In [ ]:
model.summary()

In [ ]:
video = cv2.VideoCapture(0)
    
while True:
        # Capture the video frame
        ret, img = video.read()

         # Display the resulting frame
        # to flip the video with 180 degree 
        image = cv2.flip(img, 1)
        #cv2.imshow('frame', image)
        
        # save image for prediction
        image = cv2.imwrite('Frame'+str(0)+'.jpg', image)
        image_addr = "Frame0.jpg"
        # image = cv2.imread(image_addr)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # cv2.imshow("gray", image)
        # handsModule = mediapipe.solutions.hands
        # with handsModule.Hands(static_image_mode=True) as hands:

        #         image = cv2.imread(image_addr)
        #         results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #         image_height, image_width, _ = image.shape

        #         if results.multi_hand_landmarks:
        #                 for hand_landmark in results.multi_hand_landmarks:
        #                         x = [landmark.x for landmark in hand_landmark.landmark]
        #                         y = [landmark.y for landmark in hand_landmark.landmark]

        #                         center = np.array([np.mean(x)*image_width, np.mean(y)*image_height]).astype('int32')
        #                 #     cv2.imshow('video', image)
        #                         cv2.circle(image, tuple(center), 10, (255,0,0), 1) #for checking the center
        #                         cv2.rectangle(image, (center[0]-128,center[1]-128), (center[0]+128,center[1]+128), (255,0,0), 1)
        #                         hand = image[center[1]-128:center[1]+128, center[0]-128:center[0]+128]
        #                         # hand = cv2.resize(hand, dim, interpolation = cv2.INTER_AREA)
        #                 #     cv2.imshow('video', hand)
        #                         if hand.shape==(256, 256, 3):

        #                                 image_hand = cv2.imwrite(image_addr, hand)

        # rest, thresh = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY)
        # _, contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # image = cv2.imwrite(address, thresh)
        # image = cv2.imwrite('Frame'+str(0)+'.png', image)

        # image = "Frame0.png"
        # image = cv2.cvtColor(image_addr, cv2.COLOR_BGR2GRAY)
        dim = (IMG_WIDTH, IMG_HEIGHT)
        
        # image = tf.keras.preprocessing.image.load_img(image_addr, color_mode="grayscale", target_size=dim)
        image = tf.keras.preprocessing.image.load_img(image_addr, target_size=dim)

        # Converts a PIL Image instance to a Numpy array. Return a 3D Numpy array.
        input_arr = tf.keras.preprocessing.image.img_to_array(image)
        # Convert single image to a batch.
        input_arr = np.array([input_arr])
        input_arr = input_arr.astype('float32')/255
        # input_arr = input_arr.astype('float32')



        # predict_min = input_arr.min(axis=(1, 2), keepdims=True)
        # predict_max = input_arr.max(axis=(1, 2), keepdims=True)

        # x_predict = (input_arr - predict_min) / (predict_max - predict_min)



        # Generates output predictions for the input samples. Return Numpy array(s) of predictions.
        predictions = model.predict(input_arr)
        print(predictions)
        
        # Return the index_array of the maximum values along an axis.
        pre_class = np.argmax(predictions, axis=-1)
        print(pre_class)
        #print(GESTURE[pre_class[0]])
        text = GESTURE[pre_class[0]]
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.flip(img, 1)
        if predictions[0][pre_class[0]] > 0.: 
                cv2.putText(image, 
                        text, 
                        (50, 50), 
                        font, 2,
                        (0, 0, 0), 
                        2, 
                        cv2.LINE_4)
        cv2.imshow('video', image)
    

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        k = cv2.waitKey(1)
        if k == ord('q'):
                break

video.release()       
cv2.destroyAllWindows()



In [ ]:
# import cv2
# import numpy as np
# import os
# import sys
# import tensorflow as tf
# from datetime import datetime
# from sklearn.model_selection import train_test_split
# from tensorflow.python.ops.gen_math_ops import mod

# from PIL import Image, ImageFilter

# EPOCHS = 10
# IMG_WIDTH = 640
# IMG_HEIGHT = 480
# NUM_CATEGORIES = 6
# TEST_SIZE = 0.5
# GESTURE = {0:"ok", 1:"down", 2:"up", 3:"palm", 4:"fist", 5:"l"}

# # Open image
# def load_data(data_dir):
#     """
#     Load image data from directory `data_dir`.
#     Assume `data_dir` has one directory named after each category, numbered
#     0 through NUM_CATEGORIES - 1. Inside each category directory will be some
#     number of image files.
#     Return tuple `(images, labels)`. `images` should be a list of all
#     of the images in the data directory, where each image is formatted as a
#     numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
#     be a list of integer labels, representing the categories for each of the
#     corresponding `images`.
#     """
#     images = []
#     labels = []
    
#     for dir in range(0, NUM_CATEGORIES):
#         # get path for each gesture like "/home/arpine/Desktop/data/0":  
#         d = os.path.join(data_dir, f"{str(dir)}")
#         # os.listdir(d) return the list of all names of images in that folder
#         for image_path in os.listdir(d):
#             # get the full path of specific image 
#             full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
#             image = Image.open(full_path).convert("RGB")
#             image = image.filter(ImageFilter.Kernel(
#                 size=(3, 3),
#                 kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
#                 scale=1
#             ))
#             image.save(full_path)
#             # Returns an image that is loaded from the specified file
#             image = cv2.imread(full_path, )
#             # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#             # cv2.imshow("im", image)
#             # get dimension for each image
#             dim = (IMG_WIDTH, IMG_HEIGHT)
#             # # resized the image
#             image_resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            
#              # add image and their directory name to images and labels list
#             images.append(image_resized)
#             labels.append(dir)
    
#     return images, labels
#     # print(images)


# # Filter image according to edge detection kernel
# # image = image.filter(ImageFilter.Kernel(
# #     size=(3, 3),
# #     kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
# #     scale=1
# # ))

# # # Show resulting image
# # image.show()


# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# load_data("/home/arpine/Desktop/Gesture/test")

In [ ]:
# import cv2
# NUM_CATEGORIES = 6

# GESTURE = {0:"ok", 1:"down", 2:"up", 3:"palm", 4:"fist", 5:"l"}

# def load_data(data_dir):
#     for dir in range(0, NUM_CATEGORIES):
#     # adr = "/home/arpine/Desktop/Gesture/poqr/ok/bcbf8425-d850-11eb-9ec6-0ba2456509ee.png"
#         d = os.path.join(data_dir, f"{str(dir)}")
#         # os.listdir(d) return the list of all names of images in that folder
#         for image_path in os.listdir(d):
#         # get the full path of specific image 
#             full_path = os.path.join(data_dir, f"{str(dir)}", image_path)
#             image = cv2.imread(full_path, 0)
#             rest, thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
#             _, contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#             cv2.imwrite(full_path, thresh)

In [ ]:
# full_path = "/home/arpine/Desktop/Gesture/image/carmen/ok/000000014.jpg"
# image = cv2.imread(full_path, 0)
# rest, thresh = cv2.threshold(image, 80, 80, cv2.THRESH_BINARY)
# _, contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# cv2.imwrite("frame.png", thresh)